In [1]:
import torch

See if we have cuda available (spoilers on my machine I do).

In [2]:
if torch.cuda.is_available():
    print(f'We have a {torch.cuda.device_count()} GPUs available!')
else:
    print('Sorry, CPU only.')

Sorry, CPU only.


In [3]:
my_device = 'cuda' if torch.cuda.is_available() else 'cpu'
my_device

'cpu'

In [4]:
input2 = torch.tensor([2.0], device=my_device)
input3 = torch.tensor([2.0], device=my_device)
input2+input3

tensor([4.])

Below is the [max graph python example](https://github.com/modularml/max/blob/main/tutorials/max-graph-python/src/max_ops/addition.py).

In [5]:
import numpy as np
from typing import Any
from max import engine
from max.dtype import DType
from max.graph import Graph, TensorType, ops

In [6]:
def add_tensors(a: np.ndarray, b: np.ndarray) -> dict[str, Any]:
    # 1. Build the graph
    input_type = TensorType(dtype=DType.float32, shape=(1,))
    with Graph(
        "simple_add_graph", input_types=(input_type, input_type)
    ) as graph:
        lhs, rhs = graph.inputs
        out = ops.add(lhs, rhs)
        graph.output(out)
        print("final graph:", graph)

    # 2. Create an inference session
    session = engine.InferenceSession()
    model = session.load(graph)

    for tensor in model.input_metadata:
        print(
            f"name: {tensor.name}, shape: {tensor.shape}, dtype: {tensor.dtype}"
        )

    # 3. Execute the graph
    ret = model.execute(a, b)[0]
    print("result:", ret)
    return ret

In [7]:
input0 = np.array([1.0], dtype=np.float32)
input1 = np.array([1.0], dtype=np.float32)
output = add_tensors(input0, input1)

final graph: mo.graph @simple_add_graph(%arg0: !mo.tensor<[1], f32>, %arg1: !mo.tensor<[1], f32>) -> !mo.tensor<[1], f32> attributes {argument_names = ["input0", "input1"], result_names = ["output0"]} {
  %0 = mo.chain.create()
  %1 = rmo.add(%arg0, %arg1) : (!mo.tensor<[1], f32>, !mo.tensor<[1], f32>) -> !mo.tensor<[1], f32>
  mo.output %1 : !mo.tensor<[1], f32>
}
name: input0, shape: [1], dtype: DType.float32
name: input1, shape: [1], dtype: DType.float32
result: max.driver.Tensor(DType.float32, (1,))


In [8]:
output.to_numpy()

array([2.], dtype=float32)